In [9]:
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Qt5Agg')
# %matplotlib widget

import numpy as np

In [10]:
num_x, num_y, num_z = 400, 400, 400
radius = 160

# IOR fields
center = (num_x//2, num_y//2, num_z//2)
x, y, z = np.ogrid[:num_x, :num_y, :num_z]
mask = (x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2 <= radius**2
convex_lens_ior_field = np.ones((num_x, num_y, num_z))
convex_lens_ior_field[mask] = 1.5

# concave_lens_ior_field = np.ones((num_x, num_y, num_z)) * 1.5
# concave_lens_ior_field[mask] = 1

# slope_ior_field = np.ones((num_x, num_y, num_z))
# for x in range(num_x):
#     for z in range(num_z):
#         if x > -z + 22:
#             slope_ior_field[x, :, z] = 1.5

# Initial wavefront
initial_wavefront_positions = np.array([(0, y, z) for y in range(num_y) for z in range(num_z)])
initial_wavefront_directions = np.array([(1, 0, 0) for _ in range(num_y * num_z)])

In [11]:

def show_original(ior_field):
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_box_aspect([1,1,1])
    ax.set_xlim(0, num_x)
    ax.set_ylim(0, num_y)
    ax.set_zlim(0, num_z)
    # IOR field
    x_points, y_points, z_points = np.where(ior_field == 1.5)
    ax.scatter(x_points, y_points, z_points, color='blue', alpha=0.2)

    # Initial wavefront
    ax.quiver(initial_wavefront_positions[:, 0], initial_wavefront_positions[:, 1], initial_wavefront_positions[:, 2], 
            initial_wavefront_directions[:, 0], initial_wavefront_directions[:, 1], initial_wavefront_directions[:, 2],
            length=1, color='green', normalize=True, linewidths=0.7, arrow_length_ratio=0.4)

    ax.set_title("3D Representation of a Sphere")
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')
    plt.show()

# show_original(convex_lens_ior_field)

In [13]:
def compute_3d_gradients(IOR):
    grad_x = np.gradient(IOR, axis=0)
    grad_y = np.gradient(IOR, axis=1)
    grad_z = np.gradient(IOR, axis=2)
    return grad_x, grad_y, grad_z

convex_grad_xyz = compute_3d_gradients(convex_lens_ior_field)

# grad_x, grad_y, grad_z = convex_grad_xyz
# plt.figure(figsize=(8, 3))
# plt.subplot(131)
# plt.imshow(grad_x[:, :, num_z // 2], cmap='gray')  # Z axis
# plt.title('Gradient along x axis')
# plt.xlabel('x - axis')
# plt.ylabel('y - axis')
# plt.subplot(132)
# plt.imshow(grad_y[num_y // 2, :, :], cmap='gray')  # Y axis
# plt.title('Gradient along y axis')
# plt.xlabel('x - axis')
# plt.ylabel('z - axis')
# plt.subplot(133)
# plt.imshow(grad_z[:, num_x // 2, :], cmap='gray')  # X axis
# plt.title('Gradient along z axis')
# plt.xlabel('y - axis')
# plt.ylabel('z - axis')
# plt.tight_layout()
# plt.show()

In [14]:
def update_wavefront(pos, dir, grad_xyz, IOR, delta_t):
    new_pos = []
    new_dir = []
    grad_x, grad_y, grad_z = grad_xyz
    for (x,y,z), (dx, dy, dz) in zip(pos, dir):
        # x_i+1 = x_i + delta_t * v_i / n^2
        # v_i+1 = v_i + delta_t * grad_n / n
        n = IOR[int(x), int(y), int(z)]
        new_x = x + delta_t * dx / n**2
        new_y = y + delta_t * dy / n**2
        new_z = z + delta_t * dz / n**2
        new_x = np.clip(new_x, 0, IOR.shape[0]-1)
        new_y = np.clip(new_y, 0, IOR.shape[1]-1)
        new_z = np.clip(new_z, 0, IOR.shape[2]-1)
        new_pos.append((new_x, new_y, new_z))

        n_dx = grad_x[int(x), int(y), int(z)]
        n_dy = grad_y[int(x), int(y), int(z)]
        n_dz = grad_z[int(x), int(y), int(z)]
        new_dx = dx + delta_t * n_dx / n
        new_dy = dy + delta_t * n_dy / n
        new_dz = dz + delta_t * n_dz / n
        new_dir.append((new_dx, new_dy, new_dz))

    return new_pos, new_dir

def simulate_wavefront_propagation(cur_IOR, grad_xyz, inital_wavefront_pos, initial_wavefront_dir, num_steps=100, delta_t=1.0):
    wavefront_pos_list = [inital_wavefront_pos]
    wavefront_dir_list = [initial_wavefront_dir]
    for _ in range(num_steps):
        wavefront_positions, wavefront_directions = update_wavefront(wavefront_pos_list[-1], wavefront_dir_list[-1], grad_xyz, cur_IOR, delta_t)
        wavefront_pos_list.append(wavefront_positions)
        wavefront_dir_list.append(wavefront_directions)

    return wavefront_pos_list, wavefront_dir_list

def show_propagation_results(convex_pos_res, convex_dir_res, IOR, num_show_images, num_steps, fig_name):
    num_cols = min(num_show_images, 3)
    num_rows = (num_show_images + num_cols - 1) // num_cols

    show_indices = [i for i in range(0, num_steps, num_steps // num_show_images)]
    if len(show_indices) > num_show_images:
        show_indices = show_indices[:num_show_images]

    fig = plt.figure(figsize=(6 * num_cols, 6 * num_rows))
    
    for i, index in enumerate(show_indices):
        ax = fig.add_subplot(num_rows, num_cols, i + 1, projection='3d')
        ax.set_box_aspect([1,1,1])
        ax.set_xlim(0, 20)
        ax.set_ylim(0, 20)
        ax.set_zlim(0, 20)
        ax.set_xlabel('X Axis')
        ax.set_ylabel('Y Axis')
        ax.set_zlabel('Z Axis')
        ax.set_title(f"Step {index}")
        # 3D IOR field
        x_points, y_points, z_points = np.where(IOR == 1.5)
        ax.scatter(x_points, y_points, z_points, color='blue', alpha=0.06)

        # Wavefront
        wavefront_positions = np.array(convex_pos_res[index])
        wavefront_directions = np.array(convex_dir_res[index])
        ax.quiver(wavefront_positions[:, 0], wavefront_positions[:, 1], wavefront_positions[:, 2], 
                wavefront_directions[:, 0], wavefront_directions[:, 1], wavefront_directions[:, 2],
                length=1, color='green', normalize=True, linewidths=0.7, arrow_length_ratio=0.4)
    fig.tight_layout()
    fig.suptitle(fig_name)
    plt.show()

In [ ]:
# Parameters
test_num_steps = 120
test_delta_t = 0.4
num_show_images = 6

test_list = [convex_lens_ior_field]#, concave_lens_ior_field, slope_ior_field]
test_names = ["Convex Lens", "Concave Lens", "Slope"]

for i, test in enumerate(test_list):
    grad_xyz = compute_3d_gradients(test)
    pos_res, dir_res = simulate_wavefront_propagation(test, grad_xyz, initial_wavefront_positions, initial_wavefront_directions, test_num_steps, test_delta_t)
    show_propagation_results(pos_res, dir_res, test, num_show_images, test_num_steps, test_names[i])